<a href="https://colab.research.google.com/github/antaripg/deep-learning-notebooks/blob/main/DRIVE_retinal_vessel_dataset_w_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Checking for GPU

In [ ]:
 gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.
Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


# Mounting the Google Drive

In [ ]:
from google.colab import drive

drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


### Importing the necessary Libraries

In [ ]:
%matplotlib inline
import os
import copy
import torch
import random
import numpy as np
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt

print(torch.__version__)

1.9.0+cu102


In [ ]:
Labelpath = '/content/drive/MyDrive/DRIVE_Retinal_dataset/training/1st_manual/'
Datapath = '/content/drive/MyDrive/DRIVE_Retinal_dataset/training/images/'

### Crop Images

In [ ]:
# crop random 10 x 10 patches from images and also crop corresponding label

def img_transfer(img,imgLabel, bh, bw, no_of_patch):

  h = img.shape[0]
  w = img.shape[1]
  c = img.shape[2]
  ImgArr = np.empty((no_of_patch, bh*bw*3))
  LabelArr = np.empty((no_of_patch, bh*bw*1))

  for i in range(no_of_patch):
    ih = random.randint(0, h-bh)
    iw = random.randint(0, w-bw)
    iArrI = img[ih:ih+bh,iw:iw+bw,:]
    iArrL = imgLabel[ih:ih+bh,iw:iw+bw,:]
    for ci in range(c):
      for bhi in range(bh):
        for bwi in range(bw):
          ImgArr[i][ci*bh*bw + bhi*bw + bwi] = iArrI[bhi][bwi][ci]
          if ci ==0:
            LabelArr[i][ci*bh*bw + bhi*bw + bwi] = iArrL[bhi][bwi][ci]

    return ImgArr,LabelArr


In [ ]:
patchH = 10 # height of the patch
patchW = 10 # width of the patch
PatchperImage = 1000 # no of patches per image

In [ ]:
TrainImages = torch.DoubleTensor(13*PatchperImage, 3*patchH*patchW)
TrainLabels = torch.LongTensor(13*PatchperImage, patchH*patchW)

In [ ]:
# Read each training image and crop random patches

t_no = 0
for img_no in range(13):
    imD = Image.open(Datapath + str(img_no+21) + '_training.tif')
    imD = np.array(imD)

    imL = Image.open(Labelpath + str(img_no+21) + '_manual1.gif')
    imL = np.array(imL)
    imL = np.reshape(imL, (imL.shape[0],imL.shape[1],1))

    imD,imL = img_transfer(imD,imL, patchH, patchW, PatchperImage)
    imD = imD/255.0
    imL = imL/255.0
    for i in range(PatchperImage):
        TrainImages[t_no] = torch.from_numpy(imD[i])
        TrainLabels[t_no] = torch.from_numpy(imL[i])
        t_no = t_no + 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [ ]:
# Printing Training Image and Labels size

print(TrainImages.size())
print(TrainLabels.size())

torch.Size([13000, 300])
torch.Size([13000, 100])


### Availability of GPU

In [ ]:
# Checking availability of GPU

use_gpu = torch.cuda.is_available()
if use_gpu:
  print("GPU is available!")
  device = 'cuda'
else:
  print("GPU not available!")
  device = 'cpu'

GPU not available!


### Define the AutoEncoder

In [ ]:
class autoencoder(nn.Module):
  def __init__(self):
    super(autoencoder, self).__init__()
    self.encoder = nn.Sequential(
        nn.Linear(patchH*patchW*3, patchH*patchW),
        nn.Tanh(),
        nn.Linear(patchH*patchW, patchH*patchW),
        nn.Tanh())
    self.decoder = nn.Sequential(
        nn.Linear(patchH*patchW, patchH*patchW),
        nn.Tanh(),
        nn.Linear(patchH*patchW, patchH*patchW*3),
        nn.Sigmoid())
  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

net = autoencoder()
print(net)

net = net.double().to(device)
reset = False
if reset:
  for layer in net.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
else:
  pass

init_weights = copy.deepcopy(net.encoder[0].weight.data)


autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=300, bias=True)
    (3): Sigmoid()
  )
)


NameError: ignored